In [ ]:
import numpy as np
from cloth_tools.drake.dual_ur5e import get_robot_diagram_builder_dual_ur5e
from cloth_tools.drake.collisions import get_collision_checker
from cloth_tools.stations.competition_station import CompetitionStation
from cloth_tools.ompl.state_space import single_arm_state_space
from cloth_tools.ompl.state_space import numpy_to_ompl_state
from functools import partial
from cloth_tools.ompl.state_space import function_numpy_to_ompl
from ompl import base as ob
from ompl import geometric as og
import time
from typing import List
from cloth_tools.ompl.state_space import ompl_path_to_numpy

In [ ]:
# Only run this cell once as this accesses the real hardware
station = CompetitionStation()
dual_arm = station.dual_arm

In [ ]:
current_joints_left = dual_arm.left_manipulator.get_joint_configuration()
current_joints_right = dual_arm.right_manipulator.get_joint_configuration()
current_joints = np.concatenate((current_joints_left, current_joints_right))

with np.printoptions(precision=2, suppress=True):
    print("Current joints: ", current_joints)

In [ ]:
robot_diagram_builder, robot_indexes = get_robot_diagram_builder_dual_ur5e()
plant = robot_diagram_builder.plant()
diagram = robot_diagram_builder.Build()

collision_checker = get_collision_checker(diagram, robot_indexes)

# We need a context for visualization, but it's not required if you only want to do collision checking
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
diagram.ForcedPublish(context)

In [ ]:
plant.SetPositions(plant_context, current_joints)
diagram.ForcedPublish(context)

In [ ]:
collision_checker.CheckConfigCollisionFree(current_joints)

In [ ]:
home_joints_left = np.deg2rad([180, -135, 95, -50, -90, -90])
home_joints_right = np.deg2rad([-180, -45, -95, -130, 90, 90])

home_joints = np.concatenate((home_joints_left, home_joints_right))
collision_checker.CheckConfigCollisionFree(home_joints)

In [ ]:
space = single_arm_state_space()

print(space.settings())

In [ ]:
start_state = numpy_to_ompl_state(current_joints_left, space)
goal_state = numpy_to_ompl_state(home_joints_left, space)

print(start_state)
print(goal_state)

In [ ]:
def are_joints_collision_free(joints_left, joints_right) -> bool:
    joints = np.concatenate((joints_left, joints_right))
    return collision_checker.CheckConfigCollisionFree(joints)


are_joints_collision_free(current_joints_left, current_joints_right)

In [ ]:
are_joints_left_collision_free = partial(are_joints_collision_free, joints_right=current_joints_right)
are_joints_left_collision_free(current_joints_left)

In [ ]:
is_state_valid = function_numpy_to_ompl(are_joints_left_collision_free, 6)
is_state_valid(start_state), is_state_valid(goal_state)

In [ ]:
simple_setup = og.SimpleSetup(space)
simple_setup.setStateValidityChecker(ob.StateValidityCheckerFn(is_state_valid))
simple_setup.setStartAndGoalStates(start_state, goal_state)

# TODO: Should investigateeffect of this further
step = float(np.deg2rad(5))
resolution = step / space.getMaximumExtent()
simple_setup.getSpaceInformation().setStateValidityCheckingResolution(resolution)

# Set planner to RRTstar because it keep looking for a better solutions in the given time
planner = og.RRTstar(simple_setup.getSpaceInformation())
simple_setup.setPlanner(planner)

In [ ]:
simple_setup.solve(10.0)

n_interpolated_joints = 100

if simple_setup.haveSolutionPath():
    simple_setup.simplifySolution()
    simple_setup.getSolutionPath().interpolate(n_interpolated_joints)

solution_path_left_to_home = simple_setup.getSolutionPath()

In [ ]:
def add_right_arm_joints(path: List[np.ndarray], joints_right: np.ndarray) -> List[np.ndarray]:
    path = [np.concatenate((joints, joints_right)) for joints in path]
    return path


def add_left_arm_joints(path: List[np.ndarray], joints_left: np.ndarray) -> List[np.ndarray]:
    path = [np.concatenate((joints_left, joints)) for joints in path]
    return path


path_left = ompl_path_to_numpy(solution_path_left_to_home, 6)
path = add_right_arm_joints(path_left, current_joints_right)

path_left_to_home = path


def publish_path(path: List[np.ndarray], total_time: float):
    for joints in path:
        plant.SetPositions(plant_context, joints)
        diagram.ForcedPublish(context)
        time.sleep(total_time / len(path))


publish_path(path_left_to_home, 10.0)

In [ ]:
original_joints_left = current_joints_left.copy()

start_state_new = numpy_to_ompl_state(home_joints_left, space)
goal_state_new = numpy_to_ompl_state(original_joints_left, space)

joints_new = np.concatenate((home_joints_left, current_joints_right))
plant.SetPositions(plant_context, joints_new)
diagram.ForcedPublish(context)

In [ ]:
simple_setup.clear()
simple_setup.setStartAndGoalStates(start_state_new, goal_state_new)

simple_setup.solve(10.0)

if simple_setup.haveSolutionPath():
    simple_setup.simplifySolution()
    simple_setup.getSolutionPath().interpolate(n_interpolated_joints)

solution_path_left_to_original = simple_setup.getSolutionPath()

In [ ]:
path_left = ompl_path_to_numpy(solution_path_left_to_original, 6)
path_left_to_original = add_right_arm_joints(path_left, current_joints_right)

publish_path(path_left_to_original, 10.0)

In [ ]:
def execute_joint_path_left(path: List[np.ndarray], total_time: float):
    for joints in path:
        duration = total_time / len(path)
        dual_arm.left_manipulator.servo_to_joint_configuration(joints[:6], duration).wait()


execute_joint_path_left(path_left_to_home, 20.0)

In [ ]:
execute_joint_path_left(path_left_to_original, 20.0)

In [ ]:
execute_joint_path_left(path_left_to_home, 20.0)
execute_joint_path_left(path_left_to_original, 20.0)